In [1]:
#IMPORT DES LIBRAIRIES

import os
from PIL import Image
import numpy as np
import gzip
import io

In [2]:
#INITIALISATION DES NOMS DE FICHIERS

input_dir = "2_Compressed"
output_dir = "3_Restored"

file_list = os.listdir(input_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
# PARCOURT DE CHAQUE FICHIER
for filename in file_list:
    #LECTURE DE L'IMAGE COMPRESSEE
    img_path = os.path.join(input_dir, filename)
    simple_small_img = Image.open(img_path)
    
    #CONVERSION DE L'IMAGE EN BYTES
    data = np.array(simple_small_img)
    compressed_bytes = data.tobytes()
    
    #ON DECOMPRESSE LES BYTES
    decompressed_data = gzip.decompress(compressed_bytes)
    
    #ON REAJUSTE LES DONNEES RESTAUREES

    separator_index = decompressed_data.index(b"\x00")
    filename_bytes = decompressed_data[:separator_index]
    filename = filename_bytes.replace(b"\x00", b"").decode("utf-8", "ignore") 
    file_bytes = decompressed_data[separator_index+1:]
    
    #SUPPRESSION DES CARACTERES NULS EN TROP
    if file_bytes.endswith(b"\x00"):
        file_bytes = file_bytes[:-1]
    
    #RESTAURATION DES FICHIERS D'ORIGINE
    with open(os.path.join(output_dir, filename), 'wb') as file:
        file.write(io.BytesIO(file_bytes).read())